<a href="https://colab.research.google.com/github/vandalt/phy3051-students/blob/main/tp12-deep-dreams/deep_dreams_are_made_of_these_blank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explorations des représentations dans un CNN

Avant de plonger dans les modèles génératifs, je me disais qu'il serait intéressant de réviser certains concepts avec les CNNs, notamment les représentations que le réseau se fait d'une image.

Le notebook fonctionne sur CPU, mais est un peu lent. Je suggère d'utiliser un runtime Colab avec GPU.
Si seulement des CPUs sont disponibles, vous pouvez réduire le nombre d'itérations pour accélérer le debugging.

In [ ]:
import torch
from torchvision import models, transforms

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

## Importation du modèle

Comme au TP sur les VIT, on peut utiliser un modèle PyTorch pré-entraîné.
On peut tester deux réseaux, VGG et AlexNet.
N'hésitez pas à explorer d'autres architectures également.

In [ ]:
model_name = "vgg19"  # vgg19, alexnet

if model_name == "alexnet":
    weights = models.AlexNet_Weights.DEFAULT
    model = models.alexnet(weights=weights)
elif model_name == "vgg19":
    weights = models.VGG19_Weights.DEFAULT
    model = models.vgg19(weights=weights)
model = model.to(device)
model.eval()

**Exercice: Affichez le nombre de paramètres dans le modèle et extrayez les catégories des poids pré-entraînés. Créez deux dictionnaires: idx2label et label2idx pour convertir les indices en noms de classes et vice-versa.**

In [ ]:
# TODO: Exercice

## Importation d'une image

Comme nous avons vu au dernier TP, on peut importer une image trouvée en ligne.
Contrairement à Huggingface, avec PyTorch if faudra la télécharger manuellement.

In [ ]:
import io
import requests
from PIL import Image

image_name = "sky"  # dog ou sky

if image_name == "dog":
    url = "https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg"
elif image_name == "sky":
    url = 'https://s3.amazonaws.com/pbblogassets/uploads/2018/10/22074923/pink-sky-cover.jpg'
elif image_name == "carina":
    url = "https://www.nasa.gov/wp-content/uploads/2023/03/main_image_star-forming_region_carina_nircam_final-5mb.jpg"

def download_image(url: str) -> Image:
    r = requests.get(url)
    if r.status_code == 200:
        img = Image.open(io.BytesIO(r.content))
        return img
    else:
        r.raise_for_status()

In [ ]:
pil_img = download_image(url)

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(pil_img)
plt.show()


## Transformation et inférence

Comme au TP sur les tranformeurs, on peut d'abord tester notre modèle PyTorch sur l'image trouvée en ligne.
Il suffit de transformer l'image dans le bon format, puis de la donner au modèle.

**Exercice: extrayez les transformations PyTorch des poids pré-entraînés et transformez l'image PIL en tenseur. Testez ensuite le modèle sur cette image. Affichez les 5 meilleures probabilités et les classes associées.**

In [ ]:
# TODO: Inference

## Exploration des couches du réseau

On peut d'abord visualiser le contenu des couches internes du CNN en accédant aux couches de convolution dans la composante `features` du modèle (voir la structure plus haut).

Ceci nous permet 1) de faire passer l'image dans certaines couches pour visualiser des sorties intérmédiaires et 2) de visualiser le noyau de convolution appris par cette couche.

### Activations

Commençons par inspecter les activations du réseau.

**Exercice: En accédant aux couches de convolution du réseau (`model.features`), faites passer l'image à travers la première couche de convolution et sa fonction d'activation. Quel est le format de la sortie? Affichez l'image pour l'un des canaux (_channels_).**

**Exercice: Une fois que l'exercice ci-dessus est complété, essayer de changer le canal utilisé. Essayez également d'inspecter la sortie d'une couche plus profonde dans le réseau.**

In [ ]:
# TODO: Activations

### Noyau de convolution

**Exercice: Inspectez maintenant le ou les noyaux de convolutions de votre choix. Affichez le avec imshow. Ceci nécessite d'accéder aux paramètres de la couche de convolution et de comprendre leur format.**

In [ ]:
# TODO: Noyaux

### Mécanisme de "hooks"

Au lieu de faire passer l'image manuellement jusqu'à la N-ième couche, il est possible de définir un "hook" qui ajoutera la sortie de la couche d'intérêt à PyTorch.

On peut ainsi activer à une ou plusieurs activations intermédiaires en faisant passer l'image dans le réseau au complet.

In [ ]:
# NOTE: Pas besoin de nlayer+1 car activation est "inplace",
# donc ReLU modifie le tenseur rétroactivement
target_layer = model.features[nth_layer]
activations = {}
def hook_fn(m, i, o):
    activations["output"] = o
hook = target_layer.register_forward_hook(hook_fn)

**Exercice: Imprimez les clés du dictionnaire `activations`. Appliquez ensuite le modèle à l'image que nous avons téléchargée, puis réimprimez les clés d'`activations`. Affichez ensuite le type et le format (shape) de la valeur contenue dans le dictionnaire.**

In [ ]:
# TODO: Test hook

**Exercice: Vérifiez que les activations obtenues via le _hook_ sont égales à celles obtenues manuellement plus haut.**

In [ ]:
# TODO: Check hook

On peut ensuite supprimer le _hook_.

In [ ]:
hook.remove()

## Deep dream

Essayons maintenant d'implémenter une version simple de DeepDream avec PyTorch.
Je mets en référence à la fin du notebook quelques exemples en ligne qui m'ont été utiles pour préparer ce notebook et qui contiennent plus d'information.

Le concept du DeepDream est assez simple. On traite l'image comme les paramètres et on traite les activations comme les "données". La fonction objectif $L$ peut être, par exemple la norme des activations. Au lieu de la minimiser, on la maximise. Pour une image $x$, on a donc

$$
x \leftarrow x + \alpha \nabla_x L(a)
$$

où $\alpha$ est un hyperparamètre de taux d'apprentissage.
$L$ pourait être la norme L2 des activations d'une couche, d'un seul canal, ou encore l'activation d'une seule classe à la sortie du réseau. Nous commecerons par implémenter la norme L2 des activations d'une couche.

**Exercice: Implémetez le DeepDream en PyTorch. Utilisez l'image téléchargée comme point de départ et effectuez 20 itérations avec $\alpha = 1$ pour la couche 26 des `features`.**

<details>

<summary>Cliquez pour des indications plus détaillées</summary>

Je suggère de séparer le code en deux fonctions:

- Une fonction `deep_dream(x, model, target_layer, niter, lr=1.0)` qui:
    - Clone l'image `x`
    - Active le gradient de l'image avec `requires_grad_()`
    - Enregistre un _hook_ sur `target_layer`
    - Itère le calcul du gradient et l'ascenscion de gradient via la fonction `get_gradient()` définie ci-dessous et une mise à jour des donées. `x.data` et `gradient.data` seront utiles ici.
    - Supprime le `hook`
    - Désactive le gradient de l'image avec `requires_grad_()`
    - Retourne l'image modifiée
- Une fonction `get_gradient(x, model)` qui:
    - Remet le gradient du modèle à 0
    - Passe l'image `x` dans le modèle
    - Accède aux activations d'intérêt via le dictionnaire du _hook_
    - Calcule fonction objectif (norme des activations)
    - Fait la rétropropagation

</details>

In [ ]:
# TODO: Deep dream

La fonction `postprocess` ci-dessous fait l'inverse de la normalisation pour repasser en images RGB sur 0-255 avec des unint8.

**Exerice: Affichez le résultat**

In [ ]:
def postprocess(img_tensor: torch.Tensor):
    denorm = transforms.Compose([
        transforms.Normalize(mean = [ 0., 0., 0. ], std = [ 1/0.229, 1/0.224, 1/0.225 ]),
        transforms.Normalize(mean = [ -0.485, -0.456, -0.406 ], std = [ 1., 1., 1. ]),
    ])
    img_arr = denorm(img_tensor).detach().cpu().permute(1, 2, 0).numpy()
    img_arr = np.uint8(np.clip(img_arr, 0, 1) * 255)
    return Image.fromarray(img_arr)

dream_img = postprocess(dream_tensor)
plt.imshow(dream_img)
plt.show()

**Exercice: Modifiez votre code pour que la fonction d'activation puisse soit la norme L2 soit la valeur des activations. Modifiez la également pour que le canal de `target_layer` puisse être spécifié. Testez l'optimisation pour une des classes. Vous pouvez modifier l'image téléchargée pour `sky` et ré-exécuter le notebook.**

In [ ]:
# TODO: Deep dream optimize class

**Exercice: Ajoutez un `roll` aléatoire dans votre fonction via un argument boolean `roll`. Testez la fonction avec untaux d'apprentissage de 0.2 et 50 époques, pour la couche 26 des features.**

In [ ]:
# TODO: Roll

Un autre concept qu'on voit souvent avec le DeepDream est celui d'octaves: on répète le DeepDream en changeant la taille de l'image. Typiquement, à partir de la taille initiale, on définit une liste de puissances `n` et on répète l'opération avec:

```
new_shape = base_shape * octave_scale**n
```

**Pour `octave_scale=1.3`, explorez des puissances entre -2 et 3.**

In [ ]:
# TODO: Octave

**Exercice: Testez les octaves en tentant d'optimiser une classe.**

In [ ]:
# TODO: Octave classe

In [ ]:
# décommenter pour chercher des classes à essayer
#idx2label

**Exercice: Testez une image aléatoire comme point de départ**

In [ ]:
# TODO: Random images

## Références

- [Article de blog Google Research sur Deep Dream](https://research.google/blog/inceptionism-going-deeper-into-neural-networks/)
- [Blog sur la "Deep Visualization"](https://yosinski.com/deepvis)
- [Exemple Tensorflow avec le modèle Inception V3](https://www.tensorflow.org/tutorials/generative/deepdream)
- [Implémentation simple en PyTorch](https://github.com/juanigp/Pytorch-Deep-Dream/blob/master/Deep_Dream.ipynb)
- [Implémentation plus complexe en PyTorch](https://github.com/gordicaleksa/pytorch-deepdream/blob/master/The%20Annotated%20DeepDream.ipynb)